In [ ]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import re
from skimage import io
import tifffile
from scipy.ndimage import median_filter

# Set Paths
#codedir = '/home/groups/graylab_share/OMERO.rdsStore/engje/Data/cmIF'
codedir = os.getcwd()
rootdir = f'{codedir}'
#tiffdir = f'{rootdir}/RawImages'
qcdir = f'{rootdir}/QC'
#regdir = f'{rootdir}/RegisteredImages'
subdir = f'{rootdir}/SubtractedRegisteredImages'
segdir = f'{rootdir}/Segmentation'
#czidir = f'{rootdir}/czis'

In [ ]:
# Start Preprocessing
os.chdir('/home/groups/graylab_share/data/engje/Data/')
from mplex_image import preprocess, mpimage, cmif
os.chdir(codedir)
preprocess.cmif_mkdir([qcdir,segdir,subdir])

# List slides to be processed
ls_sample = [
 ]


In [ ]:
import scipy
from skimage.filters import unsharp_mask
import skimage
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma)
from skimage import color, morphology
from skimage.transform import rescale

In [ ]:
codedir

# image renaming

In [ ]:
os.chdir(f'{codedir}/ome/ome/')

In [ ]:
df_s = pd.read_csv(f'{codedir}/Basel_Zuri_StainingPanel.csv')
ls_drop = [27,42,47,49]
df_s.drop(ls_drop,inplace=True)
#naming errors
df_s.loc[37,'Target'] = 'Ecad'
df_s.loc[26,'Target'] = 'ER'
ls_marker = ['0ArArArAr80Di', 'utheni_10311239Ru96Di', 'utheni_10311240Ru98Di', 'utheni_10311241Ru99Di', 'utheni_10311242Ru100Di',
 'utheni_10311243Ru101Di', 'utheni_10311244Ru102Di', 'utheni_10311245Ru104Di', 'istone_1261726In113Di', '15InIn115Di',
 '134XeXe134Di', 'Histone_473968La139Di', 'Cytoker_651779Pr141Di', 'Fibrone_3281668Nd142Di', 'Cytoker_3111576Nd143Di',
 'Cytoker_971099Nd144Di', 'UnknownProteinNd145Di', 'CD68_77877Nd146Di', 'Keratin_346876Sm147Di', 'SMA_174864Nd148Di',
 'Vimenti_1921755Sm149Di', 'c-Myc_322787Nd150Di', 'c-erbB-_201487Eu151Di', 'CD3epsi_8001752Sm152Di', 'Histone_phospho_383738Eu153Di',
 'Erk12_phospho_105894Sm154Di', 'Slug_3521227Gd155Di', 'Estroge_112475Gd156Di', 'Progest_312878Gd158Di', 'p53_207736Tb159Di',
 'CD44_6967Gd160Di', 'EpCAM_2971330Dy161Di', 'CD45_71790Dy162Di', 'GATA3_117792Dy163Di', 'CD20_361077Dy164Di',
 'b-Caten_1971527Ho165Di', 'Carboni_92964Er166Di', 'E-Cadhe_1031747Er167Di', 'Ki-67_1441101Er168Di', 'EGFR_1021522Tm169Di',
 'S6_phospho_166866Er170Di', 'Sox9_483739Yb171Di', 'vWF_378871Yb172Di', 'mTOR_phospho_204417Yb173Di', 'Cytoker_98922Yb174Di',
 'panCyto_234832Lu175Di', 'cleaved_198883Yb176Di', 'Iridium_10331253Ir191Di', 'Iridium_10331254Ir193Di', '202HgHg202Di',
 '204PbPb204Di', '206PbPb206Di']
#ls_metal = [item.replace('_phospho','-phospho').replace('Ru9','Ru09').replace('Ar80','Ar080').split('Di')[0][-5::] for item in ls_marker]
ls_metal = [item.split('Di')[0][-5::].replace('rAr80','Ar80').replace('9Ru','Ru').replace('0Ru','Ru').replace('1Ru','Ru')  for item in ls_marker]
d_metal = dict(zip(ls_marker,ls_metal))

In [ ]:
#ER is Rbt IgG :/; no its not!
#Registered-R0_R0c2.R0c3.R0c4.R0c5_BC44290-146-Scene-01_c1_ORG.tif
df_result = pd.DataFrame()
se_cyto = pd.Series(['Ecad','Vimentin','CD44','CD45'])
#uint16 0 to 65535
for s_file in os.listdir():
    if s_file.find('full.tiff') > -1:
        if s_file.find('BaselTMA') > -1:
            print(s_file)
            s_slide = s_file.split('_')[7]
            s_scene = s_file.split('_')[8]
            with tifffile.TiffFile(s_file) as tif:
                array = tif.asarray()
                omexml_string = tif.ome_metadata
                ls_unk = []
                imgs_cyto = []
                for idx in range(array.shape[0]):
                    #idx=37
                    img = array[idx]
                    #dtype
                    img = np.clip(img, a_min=0, a_max=65535).astype('uint16')
                    i_begin = omexml_string.find(f'ID="Channel:0:{idx}" Name="')
                    i_end = omexml_string[i_begin:].find('" SamplesPerPixel')
                    s_marker = omexml_string[i_begin + 24:i_begin + i_end]
                    #print(idx)
                    #print(s_marker.split('_')[0])
                    s_metal = d_metal[s_marker]
                    if len(df_s[df_s.loc[:,'Metal Tag'] == s_metal].Target) == 1:
                        s_target = df_s[df_s.loc[:,'Metal Tag'] == s_metal].Target.iloc[0]
                    else:
                        #print(f'Error in target {s_metal}')
                        #print(df_s[df_s.loc[:,'Metal Tag'] == s_metal].Target)
                        s_target = s_metal
                        ls_unk.append(s_marker)
                    s_name = s_target.replace(' ','').replace('/','').replace('(KRT14)','').replace('-','')
                    #print(s_name)
                    if s_name =='DNA1':
                        dapi_scale = skimage.exposure.rescale_intensity(img,in_range=(np.quantile(img,0.03),1.5*np.quantile(img,0.99999)))
                        s_filename = f"Registered-R1_{s_name}_{s_slide}-Scene-{s_scene}_c1_ORG.tif"
                        #print(s_filename)
                    elif s_name =='DNA2':
                        s_filename = f"Registered-R2_{s_name}_{s_slide}-Scene-{s_scene}_c1_ORG.tif"
                        #print(s_filename)
                    elif se_cyto.str.contains(s_name).sum() > 0:
                        cyto_scale = skimage.exposure.rescale_intensity(img,in_range=(np.quantile(img,0.03),1.5*np.quantile(img,0.99999)))
                        imgs_cyto.append(cyto_scale)
                    if s_name.find('DNA') == -1:
                        s_filename = f"Registered-R{idx}_{s_name}_{s_slide}-Scene-{s_scene}_c2_ORG.tif"
                    #hot pixels fix
                    blurred = median_filter(img, size=2)
                    difference = img.astype('int16') - blurred.astype('int16')
                    threshold = 10*np.std(difference)
                    i_filter = sum(img > int(threshold)).sum()/img.size
                    df_result.loc[s_filename,s_name] = i_filter
                    img[img > int(threshold)] = blurred[img > int(threshold)]
                    #save
                    io.imsave(f"{subdir}/{s_filename}", img,check_contrast=False)
                    #break
                #break
                #dapis
                '''
                dapi = np.clip((dapi_scale),a_min=0,a_max=65535)
                dapi_gamma = skimage.exposure.adjust_gamma(dapi, gamma=0.4)
                
                ##pretty good for cellpose, not good enough though
                ##noisy = added_gamma[200:450,300:550]
                ##tv = denoise_tv_chambolle(noisy, weight=0.25)
                ##image_rescaled = rescale(tv, 2, anti_aliasing=False)
                ##io.imsave(f'{segdir}/test/Registered-R1_DNA1r_{s_slide}-Scene-{s_scene}_c1.png',(image_rescaled*255).astype('uint8'))
                
                #The mesmer job expects images of shape[(-1, 256, 256, 2)]. use cyto mip and dapi
                mip = np.stack(imgs_cyto).max(axis=0)
                cyto = np.clip((mip),a_min=0,a_max=65535)
                cyto_gamma = skimage.exposure.adjust_gamma(cyto, gamma=0.4)
                #de-noise        
                tv = denoise_tv_chambolle(dapi_gamma, weight=0.1)
                cyto_tv = denoise_tv_chambolle(cyto_gamma, weight=0.05)
                zdh = np.dstack((tv,cyto_tv,np.zeros(tv.shape)))
                #io.imsave(f'{segdir}/Registered-R1_DNA1.Cyto_{s_slide}-Scene-{s_scene}_c1c2.png',(zdh*255).astype('uint8'))
                '''


In [ ]:
df_result.to_csv('percent_pixels_filtered.csv')

In [ ]:
%matplotlib inline
i_max = np.quantile(img,.9998)
io.imshow(img[0:100,0:100],vmin=0, vmax=i_max)

In [ ]:
io.imshow(blurred[0:100,0:100],vmin=0, vmax=i_max)

In [ ]:
io.imshow(difference[0:100,0:100],cmap='bwr')

In [ ]:
threshold

In [ ]:
#hot pixels begone
s_file = 'BaselTMA_SP43_15.9kx19.3ky_10000x5500_10_20170928_86_111_X8Y6_105_a0_full.tiff'
idx=28
with tifffile.TiffFile(s_file) as tif:
    array = tif.asarray()
    img = array[idx]
i_max = np.quantile(img,.9998)
io.imshow(img,vmin=0, vmax=i_max)
#hot pixels begone
blurred = median_filter(img, size=2)
difference = img.astype('int16') - blurred.astype('int16')
threshold = 10*np.std(difference)
img[img > int(threshold)] = 0
fig, ax = plt.subplots()
io.imshow(img,vmin=0, vmax=i_max,ax=ax)

In [ ]:
#zdh = np.dstack((np.zeros(tv.shape),cyto_tv,tv))
#io.imshow(zdh)

## segmentation example

In [ ]:
# Download multiplex data
from deepcell.datasets import multiplex_tissue
((X_train, y_train),(X_test, y_test)) = multiplex_tissue.load_data()
from deepcell.utils.plot_utils import create_rgb_image

# create rgb overlay of image data for visualization
rgb_images = create_rgb_image(X_train, channel_colors=['green', 'blue'])

# select index for displaying
idx = 6 #3
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 3, figsize=(15, 15))
ax[0].imshow(X_train[idx, ..., 0])
ax[1].imshow(X_train[idx, ..., 1])
ax[2].imshow(rgb_images[idx, ...])

ax[0].set_title('Nuclear channel')
ax[1].set_title('Membrane channel')
ax[2].set_title('Overlay')

plt.show()

In [ ]:
#export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/extras/CUPTI #enter in terminal
from deepcell.applications import Mesmer
app = Mesmer()
print('Training Resolution:', app.model_mpp, 'microns per pixel')
segmentation_predictions = app.predict(X_train, image_mpp=0.5)
# create overlay of predictions
from deepcell.utils.plot_utils import make_outline_overlay
overlay_data = make_outline_overlay(rgb_data=rgb_images, predictions=segmentation_predictions)
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(rgb_images[idx, ...])
ax[1].imshow(overlay_data[idx, ...])
ax[0].set_title('Raw data')
ax[1].set_title('Predictions')
plt.show()

In [ ]:
segmentation_predictions_nuc = app.predict(X_train, image_mpp=0.5, compartment='nuclear')
overlay_data_nuc = make_outline_overlay(rgb_data=rgb_images, predictions=segmentation_predictions_nuc)
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(rgb_images[idx, ...])
ax[1].imshow(overlay_data_nuc[idx, ...])
ax[0].set_title('Raw data')
ax[1].set_title('Nuclear Predictions')
plt.show()
#rgb_images.shape
#X_train.shape
#rgb_images.dtype
#rgb_images.max()

# segment IMC data

In [ ]:
from skimage import io, measure, segmentation
from deepcell.applications import Mesmer
from deepcell.utils.plot_utils import create_rgb_image
import numpy as np
import pandas as pd

In [ ]:
ls_file = [#'Registered-R1_DNA1.Cyto_46-Scene-X10Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_43-Scene-X5Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_22-Scene-X12Y6_c1c2.png','Registered-R1_DNA1.Cyto_95-Scene-X14Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_213-Scene-X7Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_233-Scene-X13Y7_c1c2.png','Registered-R1_DNA1.Cyto_143-Scene-X1Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_252-Scene-X6Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_3-Scene-X13Y6_c1c2.png','Registered-R1_DNA1.Cyto_39-Scene-X1Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_239-Scene-X11Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_31-Scene-X15Y2_c1c2.png',
 #'Registered-R1_DNA1.Cyto_113-Scene-X2Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_80-Scene-X4Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_14-Scene-X1Y8_c1c2.png','Registered-R1_DNA1.Cyto_262-Scene-X10Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_229-Scene-X1Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_17-Scene-X12Y4_c1c2.png',
 #'Registered-R1_DNA1.Cyto_183-Scene-X3Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_83-Scene-X13Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_282-Scene-X2Y3_c1c2.png','Registered-R1_DNA1.Cyto_68-Scene-X9Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_96-Scene-X9Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_117-Scene-X13Y3_c1c2.png',
 #'Registered-R1_DNA1.Cyto_57-Scene-X8Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_238-Scene-X7Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_108-Scene-X13Y8_c1c2.png','Registered-R1_DNA1.Cyto_68-Scene-X10Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_111-Scene-X8Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_80-Scene-X3Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_231-Scene-X6Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_222-Scene-X7Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_174-Scene-X9Y6_c1c2.png','Registered-R1_DNA1.Cyto_8-Scene-X5Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_242-Scene-X5Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_20-Scene-X12Y5_c1c2.png',
 #'Registered-R1_DNA1.Cyto_154-Scene-X8Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_137-Scene-X9Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_180-Scene-X6Y2_c1c2.png','Registered-R1_DNA1.Cyto_235-Scene-X11Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_133-Scene-X3Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_99-Scene-X5Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_61-Scene-X4Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_47-Scene-X16Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_143-Scene-X2Y2_c1c2.png','Registered-R1_DNA1.Cyto_156-Scene-X16Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_169-Scene-X3Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_49-Scene-X1Y5_c1c2.png',
 #'Registered-R1_DNA1.Cyto_141-Scene-X11Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_11-Scene-X13Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_215-Scene-X4Y1_c1c2.png','Registered-R1_DNA1.Cyto_131-Scene-X13Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_164-Scene-X9Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_117-Scene-X14Y3_c1c2.png','Registered-R1_DNA1.Cyto_136-Scene-X9Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_34-Scene-X12Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_250-Scene-X2Y3_c1c2.png','Registered-R1_DNA1.Cyto_69-Scene-X5Y3_c1c2.png','Registered-R1_DNA1.Cyto_239-Scene-X12Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_243-Scene-X14Y3_c1c2.png',
 #'Registered-R1_DNA1.Cyto_220-Scene-X10Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_162-Scene-X16Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_32-Scene-X4Y6_c1c2.png','Registered-R1_DNA1.Cyto_238-Scene-X8Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_91-Scene-X12Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_98-Scene-X11Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_185-Scene-X7Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_172-Scene-X9Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_224-Scene-X3Y7_c1c2.png','Registered-R1_DNA1.Cyto_95-Scene-X13Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_186-Scene-X6Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_158-Scene-X12Y6_c1c2.png',
 #'Registered-R1_DNA1.Cyto_173-Scene-X2Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_44-Scene-X2Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_45-Scene-X3Y3_c1c2.png','Registered-R1_DNA1.Cyto_267-Scene-X13Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_225-Scene-X2Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_92-Scene-X12Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_40-Scene-X16Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_187-Scene-X3Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_25-Scene-X4Y2_c1c2.png','Registered-R1_DNA1.Cyto_120-Scene-X6Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_67-Scene-X14Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_250-Scene-X1Y3_c1c2.png',
 #'Registered-R1_DNA1.Cyto_196-Scene-X5Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_211-Scene-X4Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_74-Scene-X15Y3_c1c2.png','Registered-R1_DNA1.Cyto_105-Scene-X9Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_264-Scene-X14Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_64-Scene-X14Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_29-Scene-X7Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_64-Scene-X13Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_61-Scene-X3Y4_c1c2.png','Registered-R1_DNA1.Cyto_116-Scene-X4Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_152-Scene-X15Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_151-Scene-X14Y7_c1c2.png',
 #'Registered-R1_DNA1.Cyto_119-Scene-X9Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_104-Scene-X7Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_103-Scene-X6Y4_c1c2.png','Registered-R1_DNA1.Cyto_136-Scene-X10Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_274-Scene-X13Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_275-Scene-X5Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_209-Scene-X11Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_23-Scene-X15Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_84-Scene-X8Y1_c1c2.png','Registered-R1_DNA1.Cyto_54-Scene-X12Y5_c1c2.png','Registered-R1_DNA1.Cyto_77-Scene-X3Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_167-Scene-X14Y8_c1c2.png',
 #'Registered-R1_DNA1.Cyto_260-Scene-X9Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_81-Scene-X11Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_78-Scene-X3Y9_c1c2.png','Registered-R1_DNA1.Cyto_142-Scene-X5Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_205-Scene-X3Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_85-Scene-X9Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_274-Scene-X14Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_283-Scene-X2Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_90-Scene-X7Y6_c1c2.png','Registered-R1_DNA1.Cyto_218-Scene-X10Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_155-Scene-X4Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_4-Scene-X15Y3_c1c2.png',
 #'Registered-R1_DNA1.Cyto_10-Scene-X1Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_132-Scene-X14Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_4-Scene-X16Y3_c1c2.png','Registered-R1_DNA1.Cyto_134-Scene-X3Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_160-Scene-X13Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_241-Scene-X8Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_56-Scene-X2Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_280-Scene-X9Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_246-Scene-X2Y6_c1c2.png','Registered-R1_DNA1.Cyto_70-Scene-X6Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_6-Scene-X12Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_271-Scene-X11Y8_c1c2.png',
 #'Registered-R1_DNA1.Cyto_263-Scene-X1Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_255-Scene-X12Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_278-Scene-X4Y2_c1c2.png','Registered-R1_DNA1.Cyto_209-Scene-X12Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_284-Scene-X8Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_16-Scene-X4Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_112-Scene-X5Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_186-Scene-X5Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_144-Scene-X16Y1_c1c2.png','Registered-R1_DNA1.Cyto_261-Scene-X11Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_172-Scene-X10Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_165-Scene-X5Y6_c1c2.png',
 #'Registered-R1_DNA1.Cyto_148-Scene-X5Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_19-Scene-X16Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_244-Scene-X3Y1_c1c2.png','Registered-R1_DNA1.Cyto_106-Scene-X2Y9_c1c2.png','Registered-R1_DNA1.Cyto_2-Scene-X2Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_21-Scene-X3Y5_c1c2.png','Registered-R1_DNA1.Cyto_141-Scene-X12Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_258-Scene-X4Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_17-Scene-X11Y4_c1c2.png','Registered-R1_DNA1.Cyto_215-Scene-X3Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_156-Scene-X15Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_228-Scene-X7Y3_c1c2.png',
 #'Registered-R1_DNA1.Cyto_214-Scene-X15Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_171-Scene-X2Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_5-Scene-X8Y7_c1c2.png','Registered-R1_DNA1.Cyto_37-Scene-X7Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_168-Scene-X10Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_59-Scene-X3Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_270-Scene-X6Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_55-Scene-X10Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_96-Scene-X10Y3_c1c2.png','Registered-R1_DNA1.Cyto_33-Scene-X14Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_170-Scene-X3Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_19-Scene-X15Y2_c1c2.png',
 #'Registered-R1_DNA1.Cyto_254-Scene-X11Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_53-Scene-X6Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_145-Scene-X2Y4_c1c2.png','Registered-R1_DNA1.Cyto_197-Scene-X4Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_142-Scene-X6Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_210-Scene-X4Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_122-Scene-X12Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_170-Scene-X4Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_72-Scene-X12Y1_c1c2.png','Registered-R1_DNA1.Cyto_192-Scene-X8Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_135-Scene-X8Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_127-Scene-X2Y8_c1c2.png',
 #'Registered-R1_DNA1.Cyto_244-Scene-X4Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_84-Scene-X7Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_130-Scene-X1Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_206-Scene-X1Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_245-Scene-X6Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_176-Scene-X3Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_79-Scene-X7Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_236-Scene-X11Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_62-Scene-X7Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_36-Scene-X5Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_277-Scene-X12Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_279-Scene-X14Y6_c1c2.png',
 #'Registered-R1_DNA1.Cyto_201-Scene-X14Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_104-Scene-X8Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_284-Scene-X7Y2_c1c2.png','Registered-R1_DNA1.Cyto_83-Scene-X14Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_237-Scene-X16Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_195-Scene-X11Y6_c1c2.png','Registered-R1_DNA1.Cyto_107-Scene-X4Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_230-Scene-X11Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_16-Scene-X3Y4_c1c2.png','Registered-R1_DNA1.Cyto_15-Scene-X1Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_13-Scene-X5Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_285-Scene-X7Y7_c1c2.png',
 #'Registered-R1_DNA1.Cyto_259-Scene-X10Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_257-Scene-X3Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_199-Scene-X8Y5_c1c2.png','Registered-R1_DNA1.Cyto_137-Scene-X10Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_253-Scene-X14Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_50-Scene-X8Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_260-Scene-X10Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_159-Scene-X11Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_38-Scene-X4Y7_c1c2.png','Registered-R1_DNA1.Cyto_128-Scene-X16Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_82-Scene-X2Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_88-Scene-X11Y5_c1c2.png',
 #'Registered-R1_DNA1.Cyto_278-Scene-X3Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_116-Scene-X3Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_91-Scene-X11Y3_c1c2.png','Registered-R1_DNA1.Cyto_269-Scene-X5Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_144-Scene-X15Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_207-Scene-X1Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_105-Scene-X10Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_150-Scene-X6Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_42-Scene-X14Y5_c1c2.png','Registered-R1_DNA1.Cyto_86-Scene-X15Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_60-Scene-X11Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_28-Scene-X11Y2_c1c2.png',
 #'Registered-R1_DNA1.Cyto_227-Scene-X9Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_50-Scene-X7Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_30-Scene-X8Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_226-Scene-X6Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_138-Scene-X3Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_7-Scene-X5Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_191-Scene-X15Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_163-Scene-X1Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_276-Scene-X9Y8_c1c2.png','Registered-R1_DNA1.Cyto_281-Scene-X8Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_256-Scene-X9Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_235-Scene-X12Y2_c1c2.png',
 #'Registered-R1_DNA1.Cyto_26-Scene-X16Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_129-Scene-X7Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_150-Scene-X5Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_190-Scene-X9Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_273-Scene-X6Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_175-Scene-X15Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_124-Scene-X10Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_223-Scene-X9Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_189-Scene-X5Y5_c1c2.png','Registered-R1_DNA1.Cyto_89-Scene-X4Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_52-Scene-X9Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_206-Scene-X2Y2_c1c2.png',
 #'Registered-R1_DNA1.Cyto_140-Scene-X10Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_184-Scene-X6Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_251-Scene-X16Y2_c1c2.png','Registered-R1_DNA1.Cyto_182-Scene-X1Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_180-Scene-X5Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_268-Scene-X6Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_178-Scene-X13Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_203-Scene-X8Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_72-Scene-X11Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_222-Scene-X8Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_75-Scene-X2Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_9-Scene-X6Y9_c1c2.png',
 #'Registered-R1_DNA1.Cyto_100-Scene-X15Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_251-Scene-X15Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_247-Scene-X13Y6_c1c2.png','Registered-R1_DNA1.Cyto_41-Scene-X11Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_282-Scene-X1Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_263-Scene-X2Y4_c1c2.png','Registered-R1_DNA1.Cyto_35-Scene-X16Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_115-Scene-X4Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_58-Scene-X16Y1_c1c2.png','Registered-R1_DNA1.Cyto_218-Scene-X9Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_139-Scene-X1Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_159-Scene-X12Y4_c1c2.png',
 #'Registered-R1_DNA1.Cyto_123-Scene-X8Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_79-Scene-X8Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_28-Scene-X12Y2_c1c2.png','Registered-R1_DNA1.Cyto_81-Scene-X12Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_166-Scene-X15Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_85-Scene-X10Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_241-Scene-X7Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_181-Scene-X6Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_58-Scene-X15Y1_c1c2.png','Registered-R1_DNA1.Cyto_272-Scene-X12Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_217-Scene-X1Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_1-Scene-X1Y8_c1c2.png',
 #'Registered-R1_DNA1.Cyto_6-Scene-X11Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_118-Scene-X13Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_193-Scene-X9Y7_c1c2.png','Registered-R1_DNA1.Cyto_188-Scene-X10Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_240-Scene-X10Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_272-Scene-X11Y3_c1c2.png','Registered-R1_DNA1.Cyto_94-Scene-X12Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_194-Scene-X11Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_146-Scene-X16Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_132-Scene-X13Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_114-Scene-X13Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_248-Scene-X8Y8_c1c2.png',
 'Registered-R1_DNA1.Cyto_109-Scene-X10Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_45-Scene-X4Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_70-Scene-X5Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_202-Scene-X9Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_126-Scene-X14Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_74-Scene-X16Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_198-Scene-X7Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_153-Scene-X7Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_208-Scene-X15Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_154-Scene-X7Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_27-Scene-X6Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_36-Scene-X6Y1_c1c2.png',
 'Registered-R1_DNA1.Cyto_48-Scene-X14Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_177-Scene-X16Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_157-Scene-X4Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_179-Scene-X13Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_200-Scene-X9Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_71-Scene-X8Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_147-Scene-X6Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_275-Scene-X6Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_161-Scene-X13Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_204-Scene-X16Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_253-Scene-X13Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_102-Scene-X1Y4_c1c2.png',
 'Registered-R1_DNA1.Cyto_101-Scene-X10Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_24-Scene-X6Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_232-Scene-X7Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_131-Scene-X14Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_265-Scene-X14Y1_c1c2.png', 'Registered-R1_DNA1.Cyto_266-Scene-X7Y8_c1c2.png','Registered-R1_DNA1.Cyto_31-Scene-X16Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_234-Scene-X10Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_51-Scene-X1Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_103-Scene-X5Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_114-Scene-X14Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_63-Scene-X4Y9_c1c2.png',
 'Registered-R1_DNA1.Cyto_73-Scene-X12Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_249-Scene-X3Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_145-Scene-X1Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_125-Scene-X5Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_102-Scene-X2Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_120-Scene-X5Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_121-Scene-X11Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_87-Scene-X15Y4_c1c2.png', 'Registered-R1_DNA1.Cyto_110-Scene-X14Y8_c1c2.png', 'Registered-R1_DNA1.Cyto_212-Scene-X1Y9_c1c2.png', 'Registered-R1_DNA1.Cyto_25-Scene-X3Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_65-Scene-X14Y6_c1c2.png',
 'Registered-R1_DNA1.Cyto_97-Scene-X16Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_221-Scene-X2Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_217-Scene-X2Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_52-Scene-X10Y2_c1c2.png', 'Registered-R1_DNA1.Cyto_149-Scene-X10Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_24-Scene-X5Y2_c1c2.png','Registered-R1_DNA1.Cyto_243-Scene-X13Y3_c1c2.png', 'Registered-R1_DNA1.Cyto_219-Scene-X8Y7_c1c2.png', 'Registered-R1_DNA1.Cyto_216-Scene-X1Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_76-Scene-X13Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_66-Scene-X15Y6_c1c2.png', 'Registered-R1_DNA1.Cyto_12-Scene-X12Y8_c1c2.png',
 'Registered-R1_DNA1.Cyto_18-Scene-X13Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_93-Scene-X15Y5_c1c2.png', 'Registered-R1_DNA1.Cyto_240-Scene-X9Y2_c1c2.png','Registered-R1_DNA1.Cyto_187-Scene-X4Y2_c1c2.png'
 ]

In [ ]:
#loop
#labels = io.imread(f'{s_scene} nuclei{nuc_diam} - Nuclei Segmentation Basins.tif')
#cell_labels = io.imread(f'{s_scene} cell{cell_diam} - Cell Segmentation Basins.tif')
image_mpp = 1
app = Mesmer()
for s_file in ls_file:
    img = io.imread(f'{segdir}/Mesmer_Images/{s_file}')
    s_scene = s_file.split('_')[2]
    print(s_scene)
    a_test = np.expand_dims(img[:,:,0:2], axis=0)
    rgb_test = create_rgb_image(a_test, channel_colors=['blue', 'green'])
    #nuc
    test_predictions = app.predict(a_test, image_mpp=image_mpp, compartment='nuclear')
    labels = np.reshape(test_predictions[0],test_predictions[0].shape[0:2])
    io.imsave(f'{segdir}/Mesmer_Segmentation/{s_scene} nuclei{image_mpp} - Nuclei Segmentation Basins.tif',labels,check_contrast=False)
    #cell
    cell_predictions = app.predict(a_test, image_mpp=image_mpp)
    cell_labels = np.reshape(cell_predictions[0],cell_predictions[0].shape[0:2])
    #clean up and save cells
    df = pd.DataFrame(measure.regionprops_table(label_image=cell_labels,intensity_image=labels,properties=('label','area','mean_intensity')))
    ls_drop = df[df.mean_intensity==0].label
    cell_labels[np.isin(cell_labels, ls_drop)] = 0
    io.imsave(f'{segdir}/Mesmer_Segmentation/{s_scene} cell{image_mpp} - Cell Segmentation Basins.tif',cell_labels,check_contrast=False)
    #break

## match

In [ ]:
from numba import jit
import time
from skimage import io, measure, segmentation
#from deepcell.applications import Mesmer
#from deepcell.utils.plot_utils import create_rgb_image
import numpy as np
import pandas as pd
#oldcontainer,d_replace = segment.nuc_to_cell(labels,cell_labels)

@jit(nopython=True, parallel=True)
def _relabel_numba(ai_cell_labels, ai_cellid_relabel, o_dtype=np.uint32):
    '''
    version: 2021-12-00

    input:
        ai_cell_labels: (n,) one dimensional numpy array with cell labels.
        ai_cellid_relabel: (n,2) dimensional numpy array which maps original cell_id to nuc_id.
        o_dtype: data type inside the image array. standard is np.uint32,
            because 32bit tiffs can be opened with most tiff compatible software.

    output:
        ai_cell_relabeld: (n,) one dimensional numpy array with cell labels that match the nucleus.

    description:
        use numba to quickly iterate over each label and replace pixels with new pixel values.
    '''
    # handle input
    ai_cell_labels = ai_cell_labels.astype(o_dtype)
    ai_cellid_relabel = ai_cellid_relabel.astype(o_dtype)

    # do match
    ai_cell_relabeld = ai_cell_labels.copy()
    for i_cellid_cell, i_cellid_nuc in ai_cellid_relabel:
        ai_cell_relabeld[ai_cell_labels == i_cellid_cell] = i_cellid_nuc

    # output
    return(ai_cell_relabeld)

def _cell_to_nuc(ai_nuc_labels, ai_cell_labels):
    '''
    version: 2021-12-00

    input:
        ai_nuc_labels: (n,) one dimensional numpy array with nucleus labels.
        ai_cell_labels: (n,) one dimensional numpy array with cell labels.

    output:
        ai_cellid_relabel: (n,2) dimensional numpy array which maps original cell_id to nuc_id.

    description:
        associate the largest nucleus contained in each cell segmentation.
    '''
    # get cell dominant nuclei label relationship
    lli_cellid_relabel = []  # numba can't handle dictionary

    # iterate over each cell label
    for i_cellid in np.unique(ai_cell_labels):
        if i_cellid == 0:
            continue

        # find all non-zero nuclei label contained within that cell mask
        ai_nuc = ai_nuc_labels[ai_cell_labels == i_cellid]
        ai_nuc = ai_nuc[ai_nuc != 0]
        ai_nuc_unique = np.unique(ai_nuc)
        # multiple nuclei, choose largest (most common pixels, i.e. mode)
        if len(ai_nuc_unique) > 1:
            ai_value, ai_count = np.unique(ai_nuc, return_counts=True) # does not work with numba
            i_nucid_mode = ai_value[np.argmax(ai_count)]
            if (i_cellid != i_nucid_mode):
                lli_cellid_relabel.append([i_cellid, i_nucid_mode])
        # one nucleus
        elif len(ai_nuc_unique) == 1:
            i_nucid_one = ai_nuc[0]
            if (i_cellid != i_nucid_one):
                lli_cellid_relabel.append([i_cellid, i_nucid_one])
        # zero nuclei
        else:
            lli_cellid_relabel.append([i_cellid, 0])

    # output
    ai_cellid_relabel = np.array(lli_cellid_relabel) #, dtype=np.int64)
    return(ai_cellid_relabel)

def relabel_cells(labels,cell_labels):
    start = time.time()
    # ravel
    ti_nuc_shape = labels.shape
    ai_nuc_labels = labels.ravel()
    ti_cell_shape = cell_labels.shape
    ai_cell_labels = cell_labels.ravel()

    # match cell id with nuclei id
    print('get _cell_to_nuc_ listing ...')
    ai_cellid_relabel = _cell_to_nuc(ai_nuc_labels=ai_nuc_labels, ai_cell_labels=ai_cell_labels)
    
    # set minimalistic variable type
    print('running numba to relabel cell labels ...')
    if (ai_cellid_relabel.max() < 2**8):
        o_dtype = np.uint8
    elif (ai_cellid_relabel.max() < 2**16):
        o_dtype = np.uint16
    elif (ai_cellid_relabel.max() < 2**32):
        o_dtype = np.uint32
    else:
        # bue 20210810: currently, this will never happen. cellpose hardcoded the mask dtype to uint32.
        # this limits cell indexing to: 2**32 -1 = 4'294'967'295 [cell].
        o_dtype = np.uint64
    # numba re-label cell id
    ai_cell_relabeled = _relabel_numba(ai_cell_labels=ai_cell_labels, ai_cellid_relabel=ai_cellid_relabel, o_dtype=o_dtype)

    # unravel
    ai_nuc_labels.shape = ti_nuc_shape
    ai_cell_relabeled.shape = ti_cell_shape
    end = time.time()
    print(f'time elapsed: {end - start}[sec]')
    return(ai_cell_relabeled)

#features
def extract_feat(labels,intensity_image, properties=('centroid','mean_intensity','area','eccentricity')):
    ''' 
    given labels and intensity image, extract features to dataframe
    '''
    props = measure.regionprops_table(labels,intensity_image, properties=properties)
    df_prop = pd.DataFrame(props)
    return(df_prop)

def label_difference(labels,cell_labels):
    '''
    given matched nuclear and cell label IDs,return cell_labels minus labels
    '''
    overlap = cell_labels==labels
    ring_rep = cell_labels.copy()
    ring_rep[overlap] = 0
    return(ring_rep)

def index_format(df,s_scene,s_col='label'):
    df.index = [f'{s_scene.split("-Scene-")[0]}_scene{s_scene.split("-Scene-")[1].split("_")[0]}_cell{item}' for item in df.loc[:,s_col]]
    return(df)

def expand_label(labels,distance=3):
    '''
    expand the nucelar labels by a fixed number of pixels
    '''
    boundaries = segmentation.find_boundaries(labels,mode='outer') #thick
    shrunk_labels = labels.copy()
    shrunk_labels[boundaries] = 0
    background = shrunk_labels == 0
    distances, (i, j) = scipy.ndimage.distance_transform_edt(
                background, return_indices=True
            )

    grown_labels = labels.copy()
    mask = background & (distances <= distance)
    grown_labels[mask] = shrunk_labels[i[mask], j[mask]]
    ring_labels = grown_labels - shrunk_labels

    return(ring_labels, grown_labels) #shrunk_labels, grown_labels,

In [ ]:
ls_scene_done = ['10-Scene-X1Y5', '104-Scene-X8Y4', '107-Scene-X4Y7', '11-Scene-X13Y7', '113-Scene-X2Y5', '117-Scene-X13Y3',
 '117-Scene-X14Y3', '13-Scene-X5Y8', '131-Scene-X13Y4', '132-Scene-X14Y1', '134-Scene-X3Y8', '136-Scene-X9Y4', '14-Scene-X1Y8',
 '141-Scene-X11Y2', '143-Scene-X1Y2', '15-Scene-X1Y9', '16-Scene-X3Y4', '160-Scene-X13Y5', '162-Scene-X16Y7', '164-Scene-X9Y8',
 '17-Scene-X12Y4', '183-Scene-X3Y6', '195-Scene-X11Y6', '201-Scene-X14Y5', '213-Scene-X7Y7', '215-Scene-X4Y1', '22-Scene-X12Y6',
 '220-Scene-X10Y5', '229-Scene-X1Y7', '230-Scene-X11Y5', '233-Scene-X13Y7', '237-Scene-X16Y6', '239-Scene-X11Y3', '239-Scene-X12Y3',
 '241-Scene-X8Y3', '243-Scene-X14Y3', '246-Scene-X2Y6', '250-Scene-X2Y3', '252-Scene-X6Y7', '255-Scene-X12Y8', '259-Scene-X10Y3',
 '262-Scene-X10Y8', '263-Scene-X1Y4', '271-Scene-X11Y8', '280-Scene-X9Y5', '282-Scene-X2Y3', '284-Scene-X7Y2', '285-Scene-X7Y7',
 '3-Scene-X13Y6', '31-Scene-X15Y2', '34-Scene-X12Y5', '39-Scene-X1Y7', '4-Scene-X16Y3', '43-Scene-X5Y5',
 '46-Scene-X10Y7', '56-Scene-X2Y6', '57-Scene-X8Y6', '6-Scene-X12Y1', '68-Scene-X9Y1', '69-Scene-X5Y3', '70-Scene-X6Y3',
 '80-Scene-X4Y3', '83-Scene-X13Y2', '83-Scene-X14Y2', '95-Scene-X14Y2', '96-Scene-X9Y3']
#from mplex_image import segment, features
ls_scene = sorted(set([item.split(' ')[0] for item in os.listdir(f'{segdir}/Mesmer_Segmentation')])) #- set(ls_scene_done))
len(ls_scene)

In [ ]:
#matching #done
'''
for s_scene in ls_scene:
    print(s_scene)
    #s_scene = '46-Scene-X10Y7'
    labels = io.imread(f'{segdir}/Mesmer_Segmentation/{s_scene} nuclei{image_mpp} - Nuclei Segmentation Basins.tif')
    cell_labels = io.imread(f'{segdir}/Mesmer_Segmentation/{s_scene} cell{image_mpp} - Cell Segmentation Basins.tif')
    cell_relabeled = relabel_cells(labels,cell_labels)
    io.imsave(f'{segdir}/Mesmer_Segmentation/{s_scene} matchedcell{image_mpp} - Cell Segmentation Basins.tif',cell_relabeled)
    #break
print(labels.max())
print(cell_labels.max())
print(cell_relabeled.max())
'''

## extract features

In [ ]:
os.chdir(subdir)
df_img = mpimage.parse_org()

In [ ]:
image_mpp=1
df_sample = pd.DataFrame()
for s_scene in ls_scene:
    labels = io.imread(f'{segdir}/Mesmer_Segmentation/{s_scene} nuclei{image_mpp} - Nuclei Segmentation Basins.tif')
    cell_labels = io.imread(f'{segdir}/Mesmer_Segmentation/{s_scene} matchedcell{image_mpp} - Cell Segmentation Basins.tif')
    s_dapi = f'Registered-R1_DNA1.Cyto_{s_scene}_c1c2.png'
    dapi = io.imread(f'{segdir}/Mesmer_Images/{s_dapi}')[:,:,0]
    #
    df_feat = extract_feat(labels,dapi, properties=(['label','area','eccentricity','centroid']))
    df_feat.columns = [f'{item}_nuclei' for item in df_feat.columns]
    df_feat = index_format(df_feat,s_scene,'label_nuclei')   
    cyto = label_difference(labels,cell_labels)
    d_loc = {'nuclei':labels,'cell':cell_labels,'cytoplasm':cyto}
    #markers (not duplicated
    df_scene = df_img[df_img.scene==s_scene].sort_values('round_ord')
    df_scene = df_scene[(df_scene.marker!='RutheniumTetroxide') & (~df_scene.index.str.contains('Registered-R11_HistoneH3'))& (~df_scene.index.str.contains('Registered-R24_HistoneH3'))]
    #loop through markers
    for s_index in df_scene.index:
        df_marker = pd.DataFrame(index=df_feat.index)
        s_marker = df_scene.loc[s_index,'marker']
        print(f'extracting features {s_marker}')
        if s_marker == 'DAPI':
            s_marker = s_marker + f'{df_scene.loc[s_index,"rounds"].split("R")[1]}'
        intensity_image = io.imread(s_index)
        for s_loc, a_loc in d_loc.items():
            df_marker_loc = extract_feat(a_loc,intensity_image, properties=(['mean_intensity','label']))
            df_marker_loc.columns = [f'{s_marker}_{s_loc}',f'label']
            df_marker_loc = index_format(df_marker_loc,s_scene) ##  df_marker_loc.index]
            df_marker_loc.drop(f'label',axis=1,inplace=True)
            df_marker = df_marker.merge(df_marker_loc, left_index=True,right_index=True,how='left',suffixes=('',f'{s_marker}_{s_loc}'))
        #extract nas
        df_na = df_marker[df_marker.isna().any(axis=1)]
        ls_na = [int(item.split('_cell')[1]) for item in df_na.index]
        ring_labels, grown_labels = expand_label(labels,distance=3)
        ring_labels[~np.isin(ring_labels,ls_na)] = 0
        grown_labels[~np.isin(grown_labels,ls_na)] = 0
        df_marker_ring = extract_feat(ring_labels,intensity_image, properties=(['mean_intensity','label']))
        df_marker_grown = extract_feat(grown_labels,intensity_image, properties=(['mean_intensity','label']))
        df_feat_na = df_marker_ring.merge(df_marker_grown,on='label',suffixes=('_ring','_grown'))
        df_feat_na = index_format(df_feat_na,s_scene)
        #fill in nas
        #break
        df_marker.loc[df_feat_na.index,f'{s_marker}_cytoplasm'] = df_feat_na.loc[:,f'mean_intensity_ring']
        df_marker.loc[df_feat_na.index,f'{s_marker}_cell'] = df_feat_na.loc[:,f'mean_intensity_grown']   
        df_feat = df_feat.merge(df_marker, left_index=True,right_index=True,how='left',suffixes=('',f'{s_marker}'))
        #break
    df_sample = df_sample.append(df_feat)
    #break
#df_sample.to_csv('features_IMC_MeanIntensity_Centroid_Shape.csv')

## filter features 

In [ ]:
pwd

In [ ]:
os.chdir(segdir)
df_mi = pd.read_csv(('features_IMC_MeanIntensity_Centroid_Shape.csv'),index_col=0)
#df_mi = df_mi.drop(['DNA2_cell', 'DNA2_nuclei', 'DNA2_cytoplasm'],axis=1).dropna()
#df_mi = df_mi.dropna() #618961 to 618811
#150 cells are na in the markers: probaly too small for the expansion (get borders erased them)
#df_mi.isna().sum().sort_values()[-3::].index
#df_mi[df_mi.drop(['DNA2_cell', 'DNA2_nuclei', 'DNA2_cytoplasm'],axis=1).isna().ArgonDimers_cytoplasm].sort_values(by='area_nuclei')
#df_mi.drop(['DNA2_cell', 'DNA2_nuclei', 'DNA2_cytoplasm'],axis=1).dropna()
#imc segmentation had 855668/ 844498 w/o liver cells
# mesmer seg has 618961 cells, 618811 NA free

In [ ]:
#drop na df_norm is the watershed 
df_norm = pd.read_csv(f'{codedir}/data/20211221_BaselTMA_MeanIntensity.csv',index_col=0,low_memory=False)
df_norm['slide_scene'] = [item.split('_')[2] + '_' + item.split('_')[3] for item in  df_norm.index]
df_mi['slide_scene'] = [item.split('_cell')[0].replace('scene','') for item in  df_mi.index]
df_norm = df_norm[~df_norm.slide_scene.isin(['Liver_X1Y1', 'Liver_X2Y1'])]
print(set(df_norm.slide_scene) - set(df_mi.slide_scene))
print(set(df_mi.slide_scene) - set(df_norm.slide_scene))

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(4,5),dpi=200)
df_norm.groupby('slide_scene').Area.count().sort_values().plot(kind='barh',ax=ax,width=1)
ax.yaxis.set_visible(False)
ax.set_xlim(0,7000)
ax.set_title('Watershed Segmentation')

In [ ]:
fig, ax = plt.subplots(figsize=(4,5),dpi=200)
ls_index = df_norm.groupby('slide_scene').Area.count().sort_values().index
df_mi.groupby('slide_scene').area_nuclei.count().loc[ls_index].plot(kind='barh',ax=ax,width=1)
ax.yaxis.set_visible(False)
ax.set_xlim(0,7000)
ax.set_title('Mesmer Segmentation')

In [ ]:
df_ws = df_norm.groupby('slide_scene').Area.count().sort_values()
ls_index = df_norm.groupby('slide_scene').Area.count().sort_values().index
df_mes = df_mi.groupby('slide_scene').area_nuclei.count().loc[ls_index]
df_both = pd.DataFrame(data=df_ws).rename({'Area':'WS'},axis=1)
df_both['MES'] = df_mes
df_both.loc[ls_seg,'off'] = 'orange'
df_both.fillna('blue',inplace=True)
df_both.head()

In [ ]:
import seaborn as sns
from scipy import stats
fig, ax = plt.subplots(figsize=(4,3),dpi=300)
sns.regplot(x="WS", y="MES", data=df_both,ax=ax,scatter_kws={'s':5,'alpha':0.5,
                                                             'color':list(df_both['off'].values),'label':['__','b'],
                                                            })
r, pval= stats.pearsonr(df_both['WS'], df_both['MES'])
ax.set_xlabel('Watershed cell count')
ax.set_ylabel('Mesmer cell count')
ax.legend(['__','outliers'],labelcolor='orange')
ax.set_title(f'Watershed vs Mesmer\nPearson r={r:.2} p={pval:.2}')
print(pval)
plt.tight_layout()
fig.savefig(f'{qcdir}/scatterplot_Mesmer_vs_Wastershed_cell_count.png')

In [ ]:
#
df_both.loc[ls_seg_two,'two'] = 'orange'
df_both.fillna('blue',inplace=True)
fig, ax = plt.subplots(figsize=(4,3),dpi=300)
sns.regplot(x="WS", y="MES", data=df_both,ax=ax,
            scatter_kws={'s':5,'alpha':0.5,'color':list(df_both['two'].values),'label':['__','o']})
r, pval= stats.pearsonr(df_both.loc[df_both.two=='blue','WS'], df_both.loc[df_both.two=='blue','MES'])
ax.set_xlabel('Watershed cell count')
ax.legend(['__','two WS ROIS'])
ax.set_ylabel('Mesmer cell count')
ax.set_title(f'Watershed vs Mesmer\nPearson r={r:.2} p={pval:.2}')
print(pval)
plt.tight_layout()
fig.savefig(f'{qcdir}/scatterplot_Mesmer_vs_Wastershed_cell_count_two.png')

In [ ]:
print(((df_ws - df_mes)/df_ws).mean())
print(((df_ws - df_mes)/df_ws).median())
print(len(df_ws))
#16 have 50% less cells
ls_seg = ((df_ws - df_mes)/df_ws).sort_values()[-16::].index

In [ ]:
ls_seg = set(ls_seg) - set(ls_seg_two)

In [ ]:
os.chdir(codedir)
os.chdir('ome/masks')
df_ws = pd.DataFrame(index=sorted(os.listdir()))

In [ ]:
os.chdir(codedir)
os.chdir('Segmentation/Mesmer_Segmentation')
df_mes_all = pd.DataFrame(index=sorted(os.listdir()))
df_mes = df_mes_all[df_mes_all.index.str.contains('matched')]

In [ ]:
df_seg = pd.DataFrame(index=os.listdir(segdir))
df_seg.drop(['Mesmer_Segmentation','features_IMC_MeanIntensity_Centroid_Shape.csv','test'],inplace=True)
df_seg['slide'] = [item.split('-Scene-')[0].split('_')[-1] for item in df_seg.index]
df_seg['scene'] =[item.split('-Scene-')[1].split('_')[0] for item in df_seg.index]
df_seg['slide_scene'] = df_seg.slide + '_' + df_seg.scene

In [ ]:
df_mes['slide'] = [item.split('-Scene-')[0] for item in df_mes.index]
df_mes['scene'] =[item.split('-Scene-')[1].split(' ')[0] for item in df_mes.index]
df_mes['slide_scene'] = df_mes.slide + '_' + df_mes.scene

In [ ]:
#BaselTMA_SP43_10.3kx13.7ky_6500x8000_6_20170928_29_19_X15Y2_216_a0_full_maks.tiff
df_ws['slide'] = [item.split('_')[-6] for item in df_ws.index]
df_ws['scene'] =[item.split('_')[-5] for item in df_ws.index]
df_ws['slide_scene'] = df_ws.slide + '_' + df_ws.scene

In [ ]:
ls_seg_two = []
for s_seg in sorted(set(df_seg.slide_scene)):
    i_len = len(df_ws[df_ws.slide_scene==s_seg].index)
    if i_len > 1:
        ls_seg_two.append(s_seg)

In [ ]:
for s_seg in ls_seg:
    #s_seg = '104_X8Y4'
    s_file_mes = df_mes[df_mes.slide_scene==s_seg].index[0]
    #print(s_seg)
    i_len = len(df_ws[df_ws.slide_scene==s_seg].index)
    s_file_ws = df_ws[df_ws.slide_scene==s_seg].index[0]
    s_file_img = df_seg[df_seg.slide_scene==s_seg].index[0]
    matched = io.imread(f'{segdir}/Mesmer_Segmentation/{s_file_mes}')
    wshed = io.imread(f'{codedir}/ome/masks/{s_file_ws}')
    img = io.imread(f'{segdir}/{s_file_img}')
    #img = 
    fig, ax = plt.subplots(1,3,figsize=(8,3),dpi=200)
    img1 =io.imshow(matched,ax=ax[1],cmap='viridis')
    #fig.colorbar(img1, ax=ax[0])
    img2 = io.imshow(wshed,ax=ax[0],cmap='viridis')
    #fig.colorbar(img2, ax=ax[1])
    io.imshow(img,ax=ax[2])
    ax[0].set_title(f'watershed: {wshed.max()}')
    ax[1].set_title(f'Mesmer: {matched.max()}')
    ax[2].set_title('Image')
    ax[0].set_xlim(0,img.shape[1])
    ax[0].set_ylim(img.shape[0],0)
    ax[1].set_xlim(0,img.shape[1])
    ax[1].set_ylim(img.shape[0],0)
    
    fig.suptitle(s_seg,y=0.9)
    plt.tight_layout()
    fig.savefig(f'{qcdir}/segmentation_basins_{s_seg}.png')
    if i_len > 1:
        s_file_ws = df_ws[df_ws.slide_scene==s_seg].index[1]
        wshed = io.imread(f'{codedir}/ome/masks/{s_file_ws}')
        fig, ax = plt.subplots(figsize=(3,3),dpi=200)
        io.imshow(wshed,ax=ax,cmap='viridis')
    #break

### testing seg

In [ ]:
    #test_predictions = app.predict(a_test, image_mpp=image_mpp, compartment='nuclear')
    #labels = np.reshape(test_predictions[0],test_predictions[0].shape[0:2])
    #io.imsave(f'{segdir}/Mesmer_Segmentation/{s_scene} nuclei{image_mpp} - Nuclei Segmentation Basins.tif',labels,check_contrast=False)
    #cell
    #cell_predictions = app.predict(a_test, image_mpp=image_mpp)
    #cell_labels = np.reshape(cell_predictions[0],cell_predictions[0].shape[0:2])

In [ ]:
io.imshow(labels)

In [ ]:
io.imshow(cell_labels)

In [ ]:
#cleaned up
io.imshow(cell_labels)

In [ ]:
df = pd.DataFrame(measure.regionprops_table(label_image=cell_labels,intensity_image=labels,properties=('label','area','mean_intensity')))
ls_drop = df[df.mean_intensity==0].label
cell_labels[np.isin(cell_labels, ls_drop)] = 0

In [ ]:
#a_test = np.expand_dims(img[:,:,0:2], axis=0)
#a_test.shape
#rgb_test = create_rgb_image(a_test, channel_colors=['blue', 'green'])
#whole cell
#app = Mesmer()
#test_predictions = app.predict(a_test, image_mpp=1)
#io.imshow(test_predictions[0])
#np.reshape(test_predictions[0],test_predictions[0].shape[0:2])
#io.imsave(f'{segdir}/test/predictions.tiff',np.reshape(test_predictions[0],test_predictions[0].shape[0:2]))

In [ ]:
'''
overlay_test = make_outline_overlay(rgb_data=rgb_test, predictions=test_predictions)
idx=0
i_size = 200
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(rgb_test[idx, 0:i_size,0:i_size,:])
ax[1].imshow(overlay_test[idx, 0:i_size,0:i_size,:])
ax[0].set_title('Raw data')
ax[1].set_title('Predictions')
plt.show()
'''

In [ ]:
#nuclear
#test_predictions_nuc = app.predict(a_test, image_mpp=1, compartment='nuclear')
#io.imshow(test_predictions_nuc[0])
#np.reshape(test_predictions[0],test_predictions[0].shape[0:2])
#io.imsave(f'{segdir}/test/predictions_nuc.tiff',np.reshape(test_predictions_nuc[0],test_predictions_nuc[0].shape[0:2]))

In [ ]:
'''
overlay_test_nuc = make_outline_overlay(rgb_data=rgb_test, predictions=test_predictions_nuc)
idx=0
i_size = 200
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(rgb_test[idx, 0:i_size,0:i_size,:])
ax[1].imshow(overlay_test_nuc[idx, 0:i_size,0:i_size,:])
ax[0].set_title('Raw data')
ax[1].set_title('Predictions')
plt.show()
'''

In [ ]:
#napari clean up
'''
from skimage import measure
df = pd.DataFrame(measure.regionprops_table(label_image=cell_labels,intensity_image=nuc_labels,properties=('label','area','mean_intensity')))
ls_drop = df[df.mean_intensity==0].label
cell_labels[np.isin(cell_labels, ls_drop)] = 0
'''

# image processing

test differerent de-noising

In [ ]:
# also tried simple
# scipy.ndi.gaussian_filter().

In [ ]:
os.chdir(f'{segdir}/test')

In [ ]:
#a_scale = skimage.exposure.rescale_intensity(dapi,in_range=(np.quantile(dapi,0.03),1.5*np.quantile(dapi,0.9999)))
a_scale = dapi_scale

In [ ]:
io.imshow(a_scale[200:300,300:400])
plt.title('Clip .9999 quantile')

In [ ]:
io.imsave('scaled.png',a_scale[200:600,300:700])

In [ ]:
a_gamma = skimage.exposure.adjust_gamma(a_scale, gamma=0.4)

In [ ]:
io.imshow(a_gamma[200:300,300:400])
plt.title('Adjust gamma')

In [ ]:
io.imsave('gamma.png',a_gamma[200:600,300:700])

In [ ]:
a_gauss = scipy.ndimage.gaussian_filter(a_gamma,sigma=1)
io.imshow(a_gauss[200:300,300:400])
plt.title('Gaussian blurr')

In [ ]:
image = a_gamma[200:600,300:700]

In [ ]:
result_1 = unsharp_mask(image, radius=1, amount=1)
result_2 = unsharp_mask(image, radius=5, amount=2)
result_3 = unsharp_mask(image, radius=20, amount=1)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2,
                         sharex=True, sharey=True, figsize=(10, 10))
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Original image')
ax[1].imshow(result_1, cmap=plt.cm.gray)
ax[1].set_title('Enhanced image, radius=1, amount=1.0')
ax[2].imshow(result_2, cmap=plt.cm.gray)
ax[2].set_title('Enhanced image, radius=5, amount=2.0')
ax[3].imshow(result_3, cmap=plt.cm.gray)
ax[3].set_title('Enhanced image, radius=20, amount=1.0')

In [ ]:
io.imsave('result_1.png',(result_1*255).astype('uint8'))

In [ ]:
io.imsave('result_2.png',(result_2*255).astype('uint8'))

In [ ]:
io.imsave('result_3.png',(result_3*255).astype('uint8'))

In [ ]:
mip = np.stack(img_st).max(axis=0)

In [ ]:
#added = np.clip((img_st[0] + img_st[1]),a_min=0,a_max=65535)
#added_gamma = skimage.exposure.adjust_gamma(added, gamma=0.5)
#pretty good
noisy = a_gamma[200:600,300:700]
tv = denoise_tv_chambolle(noisy, weight=0.25)
#io.imsave('added_more_tv.png',(tv*255).astype('uint8'))

In [ ]:
#io.imshow(mip[200:600,300:700])

In [ ]:
#io.imsave('mip.png',mip[200:600,300:700])

In [ ]:
#mip_gamma = skimage.exposure.adjust_gamma(mip, gamma=0.5)

In [ ]:
#io.imshow(mip_gamma[200:600,300:700])

In [ ]:
#io.imsave('gmip.png',mip_gamma[200:600,300:700])

In [ ]:
#denoise
noisy = a_gamma[200:300,300:400]

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(8, 6), dpi=300,
                       sharex=True, sharey=True)
ax[0, 0].imshow(noisy,cmap='gray')
ax[0, 0].axis('off')
ax[0, 0].set_title('Noisy')
ax[0, 1].imshow(denoise_tv_chambolle(noisy, weight=0.1),cmap='gray')
ax[0, 1].axis('off')
ax[0, 1].set_title('TV (0.1)')
ax[0, 2].imshow(denoise_bilateral(noisy, sigma_color=0.05, sigma_spatial=15, 
                ),cmap='gray')
ax[0, 2].axis('off')
ax[0, 2].set_title('Bilateral (0.05)')
ax[1, 0].imshow(denoise_wavelet(noisy,  rescale_sigma=True),cmap='gray')
ax[1, 0].axis('off')
ax[1, 0].set_title('Wavelet denoising')

ax[1, 1].imshow(denoise_tv_chambolle(noisy, weight=0.2),cmap='gray')
ax[1, 1].axis('off')
ax[1, 1].set_title('TV (0.25)')
ax[1, 2].imshow(denoise_bilateral(noisy, sigma_color=.2,sigma_spatial=15, 
                ),cmap='gray')
ax[1, 2].axis('off')
ax[1, 2].set_title('Bilateral (0.2)')

fig.tight_layout()

In [ ]:
noisy = added[200:250,300:350]
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(8, 6), dpi=300,
                       sharex=True, sharey=True)
ax[0, 0].imshow(noisy)
ax[0, 0].axis('off')
ax[0, 0].set_title('Noisy')
ax[0, 1].imshow(denoise_tv_chambolle(noisy, weight=0.1))
ax[0, 1].axis('off')
ax[0, 1].set_title('TV')
ax[0, 2].imshow(denoise_bilateral(noisy, sigma_color=0.05, sigma_spatial=15,
                ))
ax[0, 2].axis('off')
ax[0, 2].set_title('Bilateral')
ax[1, 0].imshow(denoise_wavelet(noisy,  rescale_sigma=True))
ax[1, 0].axis('off')
ax[1, 0].set_title('Wavelet denoising')

ax[1, 1].imshow(denoise_tv_chambolle(noisy, weight=0.2))
ax[1, 1].axis('off')
ax[1, 1].set_title('(more) TV')
ax[1, 2].imshow(denoise_bilateral(noisy, sigma_color=0.1, sigma_spatial=15,
                ))
ax[1, 2].axis('off')
ax[1, 2].set_title('(more) Bilateral')

fig.tight_layout()

In [ ]:
added_gamma = skimage.exposure.adjust_gamma(added, gamma=0.5)
#pretty good
noisy = added_gamma[200:600,300:700]
tv = denoise_tv_chambolle(noisy, weight=0.25)
io.imsave('added_more_tv.png',(tv*255).astype('uint8'))

In [ ]:
noisy = added_gamma[200:250,300:350]
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(8, 6), dpi=300,
                       sharex=True, sharey=True)
ax[0, 0].imshow(noisy)
ax[0, 0].axis('off')
ax[0, 0].set_title('Noisy')
ax[0, 1].imshow(denoise_tv_chambolle(noisy, weight=0.1))
ax[0, 1].axis('off')
ax[0, 1].set_title('TV')
ax[0, 2].imshow(denoise_bilateral(noisy, sigma_color=0.05, sigma_spatial=15,
                ))
ax[0, 2].axis('off')
ax[0, 2].set_title('Bilateral')
ax[1, 0].imshow(denoise_wavelet(noisy,  rescale_sigma=True))
ax[1, 0].axis('off')
ax[1, 0].set_title('Wavelet denoising')

ax[1, 1].imshow(denoise_tv_chambolle(noisy, weight=0.25))
ax[1, 1].axis('off')
ax[1, 1].set_title('(more) TV')
ax[1, 2].imshow(denoise_bilateral(noisy, sigma_color=0.1, sigma_spatial=15,
                ))
ax[1, 2].axis('off')
ax[1, 2].set_title('(more) Bilateral')

fig.tight_layout()

In [ ]:
#pretty good
noisy = added_gamma[200:600,300:700]
tv = denoise_tv_chambolle(noisy, weight=0.25)
io.imsave('added_more_tv.png',(tv*255).astype('uint8'))

In [ ]:
io.imshow(noisy)

In [ ]:

footprint = morphology.disk(2)
res = morphology.white_tophat(noisy, footprint)
fig, ax = plt.subplots(ncols=3, figsize=(10, 5),dpi=300)
ax[0].set_title('Original')
ax[0].imshow(noisy, cmap='gray')
ax[1].set_title('White tophat')
ax[1].imshow(res, cmap='gray')
ax[2].set_title('Complementary')
ax[2].imshow(noisy - res, cmap='gray')

plt.show()

In [ ]:
tv = denoise_tv_chambolle(noisy, weight=0.25)
footprint = morphology.disk(2)
res = morphology.white_tophat(tv, footprint)
fig, ax = plt.subplots(ncols=3, figsize=(10, 5),dpi=300)
ax[0].set_title('Original')
ax[0].imshow(tv, cmap='gray')
ax[1].set_title('White tophat')
ax[1].imshow(res, cmap='gray')
ax[2].set_title('Complementary')
ax[2].imshow(tv - res, cmap='gray')

plt.show()

In [ ]:
noisy = added_gamma[200:300,300:400]
image = denoise_tv_chambolle(noisy, weight=0.25)
result_1 = unsharp_mask(image, radius=1, amount=1)
result_2 = unsharp_mask(image, radius=5, amount=1)
result_3 = unsharp_mask(image, radius=20, amount=1)

fig, axes = plt.subplots(nrows=2, ncols=2,
                         sharex=True, sharey=True, figsize=(10, 10))
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Original image')
ax[1].imshow(result_1, cmap=plt.cm.gray)
ax[1].set_title('Enhanced image, radius=1, amount=1.0')
ax[2].imshow(result_2, cmap=plt.cm.gray)
ax[2].set_title('Enhanced image, radius=5, amount=1.0')
ax[3].imshow(result_3, cmap=plt.cm.gray)
ax[3].set_title('Enhanced image, radius=20, amount=1.0')

In [ ]:
noisy = added_gamma[200:300,300:400]
image = denoise_tv_chambolle(noisy, weight=0.25)
result_1 = unsharp_mask(image, radius=1, amount=1)
result_2 = unsharp_mask(image, radius=1, amount=2)
result_3 = unsharp_mask(image, radius=1, amount=4)

fig, axes = plt.subplots(nrows=2, ncols=2,
                         sharex=True, sharey=True, figsize=(10, 10))
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Original image')
ax[1].imshow(result_1, cmap=plt.cm.gray)
ax[1].set_title('Enhanced image, radius=1, amount=1.0')
ax[2].imshow(result_2, cmap=plt.cm.gray)
ax[2].set_title('Enhanced image, radius=1, amount=2.0')
ax[3].imshow(result_3, cmap=plt.cm.gray)
ax[3].set_title('Enhanced image, radius=1, amount=4.0')

In [ ]:
#pretty good
noisy = added_gamma[200:600,300:700]
tv = denoise_tv_chambolle(noisy, weight=0.25)
unsharp = unsharp_mask(tv, radius=1, amount=4)
io.imsave('added_more_tv_unsharp.png',(unsharp*255).astype('uint8'))

In [ ]:
#pretty bad

tvu = denoise_tv_chambolle(unsharp, weight=0.1)
io.imsave('added_more_tv_us_tv.png',(tvu*255).astype('uint8'))